## Import Packages

In [ ]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import pandas as pd
import skimage
from operator import itemgetter
from skimage.filters.rank import entropy
from skimage.draw import disk
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.io import imread
from skimage.io import imread_collection
from scipy import ndimage as ndi
from skimage import io, feature, data, filters, measure, morphology
import plotly
import plotly.express as px
import plotly.graph_objects as go

## Constant Values, Dependent on Image Characteristics

In [ ]:
MIN_RADIUS = 94             # min radius should be bigger than noise, about the radius of the fiber end
MAX_RADIUS = 100             # max radius of circles to be detected (in pixels), about the radius of the fiber end
ACTUAL_RADIUS = 50          #known radius of the desired region of interest (ROI). This will be centered within the located circle
LARGER_THRESH = 1.2         # circle is considered significantly larger than another one if its radius is at least x bigger
OVERLAP_THRESH = 0.075        # circles are considered overlapping if this part of the smaller circle is overlapping

## Define Functions to Detect Circles

In [ ]:
def circle_overlap_percent(centers_distance, radius1, radius2):
    
    #Calculating the percentage area overlap between circles
    
    R, r = max(radius1, radius2), min(radius1, radius2)
    if centers_distance >= R + r:
        return 0.0
    elif R >= centers_distance + r:
        return 1.0
    R2, r2 = R**2, r**2
    x1 = (centers_distance**2 - R2 + r2 )/(2*centers_distance)
    x2 = abs(centers_distance - x1)
    y = math.sqrt(R2 - x1**2)
    a1 = R2 * math.atan2(y, x1) - x1*y
    if x1 <= centers_distance:
        a2 = r2 * math.atan2(y, x2) - x2*y
    else:
        a2 = math.pi * r2 - a2
    overlap_area = a1 + a2
    return overlap_area / (math.pi * r2)

def circle_overlap(c1, c2):
    d = math.sqrt((c1[0]-c2[0])**2 + (c1[1]-c2[1])**2)
    return circle_overlap_percent(d, c1[2], c2[2])

def inner_circle(cs, c, thresh):
    #Determine whether the circle 'c' is inside one circle 'cs'
    for dc in cs:
        # if new circle is larger than existing -> not inside
        if c[2] > dc[2]*LARGER_THRESH: continue
        # if new circle is smaller than existing one
        if circle_overlap(dc, c)>thresh:
            # and there is a significant overlap -> inner circle
            return True
    return False

## Detect the Circles and Plot the Output

In [ ]:
Apical = imread_collection(dirNameA) #This will update every 10 minutes to include new images
Basal = imread_collection(dirNameB)  #This will update every 10 minutes to include new images

### Both should be index 0, dependent on data set
imgA = Apical[0] #We take the baseline image from Apical as imgA 
imgB = Basal[0] #We take the baseline image from Basal as imgB

img_all = np.array([imgA, imgB])
drawn_circles_list = []
for img in img_all:
#Increase the baseline image gain to make the edges easier to locate
    adj = skimage.exposure.adjust_gamma(img, gamma=1, gain = 4)
    #img = adj #Ability to switch between either applying entropy filter to original or gain increased image
    entr_img = entropy(img, skimage.morphology.disk(15)) #Applying to chosen image
    image = entr_img #Renaming for easier variable calling in later functions
    edges = canny(image, sigma=3) #Applying canny filter to the entropy filtered image

# Detect circles of specific radii
    hough_radii = np.arange(MIN_RADIUS, MAX_RADIUS, 2) #Giving the range to look between for circles (must be 2 values for min/max)
    hough_res = hough_circle(edges, hough_radii) #Actually detect the circles from the Canny filtered image and min/max radii chosen

# Select the most prominent circles (in order from best to worst)
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii) #Ignoring noise that may look like one of the 7 circles

# Determine EXPECTED_FIBERS circles to be drawn, remove duplicate and overlapping circles from the detected max peaks
    drawn_circles = []
    for crcl in zip(cy, cx, radii):
    # Do not draw circles if they are mostly inside better fitting ones
        if not inner_circle(drawn_circles, crcl, OVERLAP_THRESH):
        # A good circle found: exclude smaller circles it covers
            i = 0
            while i<len(drawn_circles):
                if circle_overlap(crcl, drawn_circles[i]) > OVERLAP_THRESH:
                    t = drawn_circles.pop(i)
                else:
                    i += 1
        # Remember the new circle
            drawn_circles.append(crcl)
    # Stop after have found more circles than needed
        if len(drawn_circles)>EXPECTED_FIBERS:
            break

    drawn_circles = drawn_circles[:EXPECTED_FIBERS] #Stopping the appended list by the number of needed circles (7)
    for i in np.arange(len(drawn_circles)):
        drawn_circles[i] = (drawn_circles[i][0], drawn_circles[i][1], ACTUAL_RADIUS)
    
    drawn_circles_sorted = sorted(drawn_circles,key=itemgetter(1)) #rearranging the found circles to label them from left to right
    drawn_circles_list.append(drawn_circles)
# Actually draw circles
    colors  = [(250, 0, 0), (0, 250, 0), (0, 0, 250)] #Choosing between all rgb
    colors += [(200, 200, 0), (0, 200, 200), (200, 0, 200)] #Changing each circle color by values
    
    from skimage import color
    image_final = color.gray2rgb(image)
    for center_y, center_x, radius in drawn_circles_sorted:
        circy, circx = disk((center_y, center_x), ACTUAL_RADIUS) #Making the circles from stored coordinate and radius values
        color = colors.pop(0)
        image_final[circy, circx] = color
        colors.append(color)
    # Plot preprocessing results
    fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(10, 10))
#Show the imported image without any adjustment
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Original', fontsize=10)
#Show the image with gain increase
    ax[1].imshow(adj, cmap='gray')
    ax[1].set_title(r'Gain = $3.25$', fontsize=10)
#Show the result after applying the entropy filter
    ax[2].imshow(entr_img, cmap='gray')
    ax[2].set_title(r'Entropy filter, $Disk=15$', fontsize=10)
#Show the result after applying the Canny filter
    ax[3].imshow(edges, cmap='gray')
    ax[3].set_title(r'Canny filter, $\sigma=3$', fontsize=10)
    
    ax[4].imshow(img, cmap=plt.cm.gray)
    ax[4].imshow(image_final/255, cmap=plt.cm.gray, alpha = 0.15)
    ax[4].set_title(r'Circle Overlay', fontsize=10)
    
    for a in ax:
        a.axis('off')  
    fig.tight_layout()
    plt.show()
    
    print('Found', len(drawn_circles),'circles with (y,x,r):',
          drawn_circles_sorted) #Printing the coordinates and radii of the found circles as a numerical check